# Model Optimization-Hyperparameters selection
Hyperparameter: parameter that can be tuned to optimize the performance of a learning algorithm.

* How should the dataset be created to find the **optimal tuning parameter**?
* How can K-fold cross-validation be used to search for an **optimal tuning parameter**?
* How do you search for **multiple tuning parameters** at once?
* How can we combine hyperparameters tuning and cross-validation with small dataset?
* How can the **computational expense** of this process be reduced?


Parameter tuning needs to be viewed as part of the learning algorithm and must be done using the training data only. The procedure that should be followed is the one in which we: 
1) Split the training data into a smaller “training” set and a "validation set” (normally, the data is shuffled first)
2) Build models using different values of the hyperparameter
k on the new, smaller training set and evaluate them on the validation set
3) Pick the best value of k and rebuild the model on the full original training set
4) Evaluate on a separate test dataset

**Adjusting the hyperparameter to the test data will lead to optimistic performance estimates on test
data!**

In [1]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from utilities.ml_utilities import print_cv_results
import numpy as np
import pandas as pd

In [2]:
# Load the dataset and retrieve features and target
iris = load_iris()
X, y = iris.data, iris.target

## Hyperparameters and k-fold cross-validation `GridSearchCV`
* For each combination of hyperparameters $H_i$ we would like to evaluate:
    1) We fit the model $k$ times in order to validate the model on each fold.
    2) We compute the average accuracy over the $k$ fold, for a combination of hyperparameters $H_i$.
* We pick the combination of hyperparameters with the best average accuracy.
* Refit the model with the best hyperparameters on the entire training set.
* We evaluate the model on the test set.


Firstly, we split the dataset into training and test. We use the 20% to test and the remaining for training the model. We use holdout method with stratification to split the dataset into training and test.

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, stratify=y)

Firstly, define the parameter values that should be searched. Then, Create a parameter grid: map the parameter names to the values that should be searched. We are defining the search space for our model.

In [4]:
param_grid = dict(n_neighbors=[1, 3, 5, 7, 9, 12])
print(f'Search space for KNearestNeighbours:\n{param_grid}')

Search space for KNearestNeighbours:
{'n_neighbors': [1, 3, 5, 7, 9, 12]}


Instantiate the grid and start the search. **NB:**
* We select **`cv = 10`**, thus we are performing 10-fold cross-validation.
* We can set **`refit = True`** if we would like to rebuild the model on the entire training set with the best hyperparameters.
* We can set **`n_jobs = -1`** to run computations in parallel (if supported by your computer and OS).

In [5]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=10, scoring='accuracy', 
                    n_jobs=-1, refit=True, return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 12]},
             return_train_score=True, scoring='accuracy')

In [6]:
# view the complete results (list of named tuples)
#pd.DataFrame(grid.cv_results_)
#TODO

We can retrieve the best hyperparameters has followed, the best score and the best model.

In [7]:
print(f'Best validation score: {grid.best_score_}')
print(f'Best hyperparameters: {grid.best_params_}')
best_model = grid.best_estimator_
print(f'Best model: {best_model}')

Best validation score: 0.975
Best hyperparameters: {'n_neighbors': 7}
Best model: KNeighborsClassifier(n_neighbors=7)


Finally, we can evaluate our model on the test set.

In [8]:
print(f'Test accuracy score: {best_model.score(X_test, y_test)}')

Test accuracy score: 0.9666666666666667


## Searching multiple parameters simultaneously
We will see how to search multiple parameters simultaneously. In addition, we will use **`cv = RepeatedStratifiedKFold`**  as **`cv`** strategy.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, stratify=y)

In [10]:
param_grid = dict(n_neighbors=[1, 3, 5, 7, 9, 12],
                  weights=['uniform', 'distance'])
print(f'Search space for KNearestNeighbours:\n{param_grid}')

Search space for KNearestNeighbours:
{'n_neighbors': [1, 3, 5, 7, 9, 12], 'weights': ['uniform', 'distance']}


In [11]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10)

In [12]:
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, scoring='accuracy',
                    n_jobs=-1, refit=True, return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=10, random_state=None),
             estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 12],
                         'weights': ['uniform', 'distance']},
             return_train_score=True, scoring='accuracy')

In [13]:
# view the complete results (list of named tuples)
pd.DataFrame(grid.cv_results_)
#TODO

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,...,split92_train_score,split93_train_score,split94_train_score,split95_train_score,split96_train_score,split97_train_score,split98_train_score,split99_train_score,mean_train_score,std_train_score
0,0.001749,0.000698,0.007286,0.005147,1,uniform,"{'n_neighbors': 1, 'weights': 'uniform'}",0.916667,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.009065,0.023405,0.008265,0.016526,1,distance,"{'n_neighbors': 1, 'weights': 'distance'}",0.916667,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
2,0.004138,0.010204,0.014871,0.019011,3,uniform,"{'n_neighbors': 3, 'weights': 'uniform'}",0.916667,1.0,1.0,...,0.981481,0.962963,0.962963,0.972222,0.972222,0.972222,0.972222,0.962963,0.968704,0.006649
3,0.014482,0.044077,0.018549,0.054814,3,distance,"{'n_neighbors': 3, 'weights': 'distance'}",0.916667,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.002229,0.003141,0.007066,0.003851,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.833333,1.0,1.0,...,0.981481,0.953704,0.962963,0.972222,0.953704,0.981481,0.953704,0.962963,0.966481,0.008956
5,0.002929,0.005029,0.005017,0.008285,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.833333,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
6,0.002249,0.002853,0.007586,0.005110,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.833333,1.0,1.0,...,0.981481,0.962963,0.972222,0.990741,0.972222,0.972222,0.972222,0.962963,0.973611,0.007565
7,0.001909,0.000917,0.003608,0.001953,7,distance,"{'n_neighbors': 7, 'weights': 'distance'}",0.833333,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
8,0.002179,0.002608,0.007016,0.003309,9,uniform,"{'n_neighbors': 9, 'weights': 'uniform'}",0.916667,1.0,1.0,...,0.981481,0.972222,0.972222,0.972222,0.972222,0.990741,0.962963,0.972222,0.973426,0.007709
9,0.002239,0.001989,0.004088,0.003279,9,distance,"{'n_neighbors': 9, 'weights': 'distance'}",0.916667,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [14]:
print(f'Best validation score: {grid.best_score_}')
print(f'Best hyperparameters: {grid.best_params_}')
best_model = grid.best_estimator_
print(f'Best model: {best_model}')

Best validation score: 0.9758333333333333
Best hyperparameters: {'n_neighbors': 12, 'weights': 'distance'}
Best model: KNeighborsClassifier(n_neighbors=12, weights='distance')


In [15]:
print(f'Test accuracy score: {best_model.score(X_test, y_test)}')

Test accuracy score: 1.0


## What to do when the training sets are very small? `nested cross-validation`

## Reducing computational expense using `RandomizedSearchCV`
- Searching many different parameters at once may be computationally infeasible
- `RandomizedSearchCV` searches a subset of the parameters, and you control the computational "budget"

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, stratify=y)

In [17]:
from sklearn.model_selection import RandomizedSearchCV
# specify "parameter distributions" rather than a "parameter grid"
param_dist = dict(n_neighbors=range(1,12), weights=['uniform', 'distance'])
print(f'Search space for KNearestNeighbours:\n{param_dist}')

Search space for KNearestNeighbours:
{'n_neighbors': range(1, 12), 'weights': ['uniform', 'distance']}


In [18]:
# n_iter controls the number of searches
rand = RandomizedSearchCV(KNeighborsClassifier(), param_dist, cv=10, scoring='accuracy', n_iter=10, refit=True, return_train_score=True)
rand.fit(X, y)

RandomizedSearchCV(cv=10, estimator=KNeighborsClassifier(),
                   param_distributions={'n_neighbors': range(1, 12),
                                        'weights': ['uniform', 'distance']},
                   return_train_score=True, scoring='accuracy')

In [19]:
# view the complete results (list of named tuples)
pd.DataFrame(rand.cv_results_)
#TODO

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_weights,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0.001500,1.023912e-03,0.004794,0.002179,uniform,8,"{'weights': 'uniform', 'n_neighbors': 8}",1.0,0.933333,1.0,...,0.977778,0.985185,0.977778,0.977778,0.992593,0.977778,0.977778,0.977778,0.980000,0.005785
1,0.001300,7.808411e-04,0.004597,0.000800,uniform,9,"{'weights': 'uniform', 'n_neighbors': 9}",1.0,0.933333,1.0,...,0.977778,0.985185,0.977778,0.977778,0.992593,0.970370,0.977778,0.977778,0.979259,0.006458
2,0.001099,3.008771e-04,0.002199,0.000399,distance,3,"{'weights': 'distance', 'n_neighbors': 3}",1.0,0.933333,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
3,0.000901,3.004388e-04,0.003397,0.000664,uniform,1,"{'weights': 'uniform', 'n_neighbors': 1}",1.0,0.933333,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.001099,2.998432e-04,0.001599,0.000490,distance,11,"{'weights': 'distance', 'n_neighbors': 11}",1.0,0.933333,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
5,0.001099,2.999915e-04,0.002000,0.000775,distance,10,"{'weights': 'distance', 'n_neighbors': 10}",1.0,0.933333,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
6,0.000899,5.378978e-04,0.001999,0.000632,distance,1,"{'weights': 'distance', 'n_neighbors': 1}",1.0,0.933333,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
7,0.000799,3.997452e-04,0.001500,0.000500,distance,9,"{'weights': 'distance', 'n_neighbors': 9}",1.0,0.933333,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
8,0.000901,3.004855e-04,0.001397,0.000488,distance,6,"{'weights': 'distance', 'n_neighbors': 6}",1.0,0.933333,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
9,0.000999,8.730809e-07,0.003098,0.000300,uniform,6,"{'weights': 'uniform', 'n_neighbors': 6}",1.0,0.933333,1.0,...,0.970370,0.977778,0.985185,0.955556,0.985185,0.970370,0.970370,0.970370,0.972593,0.008148


In [20]:
print(f'Best validation score: {rand.best_score_}')
print(f'Best hyperparameters: {rand.best_params_}')
best_model = rand.best_estimator_
print(f'Best model: {best_model}')

Best validation score: 0.9733333333333334
Best hyperparameters: {'weights': 'uniform', 'n_neighbors': 9}
Best model: KNeighborsClassifier(n_neighbors=9)


In [21]:
print(f'Test accuracy score: {best_model.score(X_test, y_test)}')

Test accuracy score: 1.0
